In [0]:
%run ./my_packages

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


# Daily functions

# Weekly functions

# Monthly functions

In [0]:
# Set `mediatel_date` here if you want to run manually. Otherwise under BAU conditions (i.e. when you don't need to run any past dates) this cell should be skipped via %skip tag.
start = '20260103'
end = '20260201'
mediatel_dates = [
    (datetime.strptime(start, "%Y%m%d") + timedelta(days=i)).strftime("%Y%m%d") for i in range((datetime.strptime(end, "%Y%m%d") - datetime.strptime(start, "%Y%m%d")).days + 1)
    ]

In [0]:
for mediatel_date in mediatel_dates:
    # Logic to guide whether to trigger summary or append. This can be done by comparing the week number of latest build up file with the week number of latest Mediatel extract.
    # Read in latest Mediatel data and transform
    df_mediatel = f_LoadLatestMediatel(mediatel_date, select_cols=select_cols)
    df_mediatel = f_MonthlyCreateMonthNum(df_mediatel)

    # Read in current buildup file else create one if it doesn't exist
    buildup_file_name = "./Output/brett_collections_funnel_buildup_monthly.parquet"
    if os.path.isfile(buildup_file_name): 
        # Buildup file exists; load it
        df_buildup = pl.read_parquet(buildup_file_name)
    else:
        # Buildup file doesn't exist; use df_mediatel to initiate it.
        # Realistically this else block should never run since there should always be a buildup file that exists.
        df_mediatel.write_parquet(buildup_file_name)
        df_builldup = df_mediatel
        # Exit try block here so that buildup file doesn't get duplicated by going into `call_month_buildup == call_month_mediatel` path.
        sys.exit(0)
    # Compare week numbers between latest Mediatel data and buildup file
    call_month_mediatel = df_mediatel.select("call_month").max().item()
    call_month_buildup = df_buildup.select("call_month").max().item()
    if call_month_buildup == call_month_mediatel:
        # Continue building up temporary buildup table
        print("Month between buildup table and latest Mediatel extract is same. Append to build-up table and overwrite.")
        # Stack latest mediatel data to buildup table        
        df_buildup = pl.concat([df_buildup, df_mediatel], how="vertical")
        # Overwrite buildup table
        df_buildup.write_parquet(buildup_file_name)

    else:
        # Summarise buildup table, append to weekly summary, then re-create temporary table.
        print("New month start. Trigger append process then re-create temporary table.")
        # Summarise buildup table
        # Metrics should only be created at the end of each monthy prior to appending to main monthly summary table
        df_buildup = f_CreateMetrics(df_buildup, timeframe="monthly")
        df_buildup_summary = f_CreateSummary(df_buildup, "monthly")
        df_buildup_summary = df_buildup_summary.to_pandas()
        # Append now summarised buildup table to weekly summary
        table_name = "teams.cashalo.brett_collections_funnel_monthly"
        monthly_summary = spark.sql(f"SELECT * FROM {table_name}").toPandas()
        monthly_summary = pd.concat([monthly_summary, df_buildup_summary], ignore_index=True)
        # Save new weekly summary
        spark_df = f_MonthlyCreateSparkTable(monthly_summary, table_name=table_name)
        spark_df.write.saveAsTable(f"{table_name}")
        # Overwrite buildup table using latest mediatel data
        df_mediatel.write_parquet(buildup_file_name)

Month between buildup table and latest Mediatel extract is same. Append to build-up table and overwrite.
Month between buildup table and latest Mediatel extract is same. Append to build-up table and overwrite.
Month between buildup table and latest Mediatel extract is same. Append to build-up table and overwrite.
Month between buildup table and latest Mediatel extract is same. Append to build-up table and overwrite.
Month between buildup table and latest Mediatel extract is same. Append to build-up table and overwrite.
Month between buildup table and latest Mediatel extract is same. Append to build-up table and overwrite.
Month between buildup table and latest Mediatel extract is same. Append to build-up table and overwrite.
Month between buildup table and latest Mediatel extract is same. Append to build-up table and overwrite.
Month between buildup table and latest Mediatel extract is same. Append to build-up table and overwrite.
Month between buildup table and latest Mediatel extract

# Checks

In [0]:
%sql
-- Check Tableau data file
SELECT * FROM teams.cashalo.brett_collections_funnel_monthly

num_affected_rows,num_inserted_rows


In [0]:
# Check buildup file
pl.read_parquet("./Output/brett_collections_funnel_buildup_monthly.parquet").group_by("call_date").len().sort("call_date")

call_date,len
date,u32
2026-02-01,525941
